<a href="https://colab.research.google.com/github/ge96lip/Quantum-Computing/blob/main/QC_Shor's_Factorization_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>